In [1]:
import pandas as pd
from datetime import timedelta

df = pd.read_csv("assignment.csv")
print(type(df))

<class 'pandas.core.frame.DataFrame'>


# cleaning data

Removing rows with no entries as assuming any data does not makes sense.

Removing few columns as they are not needed for the assignment.

In [2]:
df = df.dropna()
df = df.reset_index(drop=True)
df = df.drop(['Pay Cycle Start Date','Pay Cycle End Date'],axis=1)
df['Time']=pd.to_datetime(df['Time'])
df['Time Out']=pd.to_datetime(df['Time Out'])

C:\Users\sahil\AppData\Local\Temp\ipykernel_20264\1727793595.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time Out']=pd.to_datetime(df['Time Out'])


## Employees who have worked for more than 14 hours in a single shift

In [3]:
df['Diff']=df['Time Out'] - df['Time']
df['Diff'] = df['Diff'].dt.total_seconds()/3600

def getHardWorkers():
    HardWorkers=[]
    def diff(row):
        hours = row['Diff']
        if hours>14:
            HardWorkers.append({'Name':row['Employee Name'],'Position':row['Position ID']})
    df.apply(diff,axis=1)
    return HardWorkers
print(getHardWorkers())

[{'Name': 'DeXgadiXXE REdarMe, ChrisMiaW S', 'Position': 'WFS000523'}]


## Employees who have less than 10 hours of time between shifts but greater than 1 hour of work in each shifts

** more than 1 hour shifts - less than 10 hours gap - more than 1 hour shift **

In [4]:
def is_same_employee(row1, row2):
    return row1['Employee Name'] == row2['Employee Name']

def getWorkers(df):
    workers = []
    for index, row in df.iterrows():
        if index > 0 and is_same_employee(row, df.iloc[index - 1]):
            shift_diff = row['Time'] - df.iloc[index - 1]['Time Out']
            if shift_diff <= pd.Timedelta('10 hours') and (row['Diff'] >= 1 or df.iloc[index - 1]['Diff'] >= 1):
                worker_info = {'Name': row['Employee Name'], 'Position': row['Position ID']}
                if worker_info not in workers:
                    workers.append(worker_info)
    
    return workers

for items in getWorkers(df):
    print(items)


{'Name': 'REsaXiaWE, XAis', 'Position': 'WFS000065'}
{'Name': 'Sparks, KeWWeMh', 'Position': 'WFS000101'}
{'Name': 'GaXCes, EXias XEpez', 'Position': 'WFS000127'}
{'Name': 'CaMaXaWE, CeghaW', 'Position': 'WFS000153'}
{'Name': 'Arias, FeXipe', 'Position': 'WFS000170'}
{'Name': 'Merrazas, GriaW', 'Position': 'WFS000180'}
{'Name': 'GarWes, JAsMiW', 'Position': 'WFS000181'}
{'Name': 'FeXix, XeEWeX', 'Position': 'WFS000183'}
{'Name': 'HerCEsiXXE, PedrE', 'Position': 'WFS000185'}
{'Name': 'Xee, XaCar', 'Position': 'WFS000200'}
{'Name': 'GaWks, GryaW', 'Position': 'WFS000226'}
{'Name': 'MAake, CasiaWE', 'Position': 'WFS000228'}
{'Name': 'SiWgh, MarAWpreeM', 'Position': 'WFS000246'}
{'Name': 'DiWh, DaA', 'Position': 'WFS000268'}
{'Name': 'ArEsMigAi, AXexaWder', 'Position': 'WFS000306'}
{'Name': 'De Xa Cerda, IgWaciE', 'Position': 'WFS000318'}
{'Name': 'REdrigAez, PedrE', 'Position': 'WFS000336'}
{'Name': 'MraW, WiXXiaC Ha', 'Position': 'WFS000345'}
{'Name': 'MAcker, CichaeX', 'Position': 'WFS0

## Worker who has worked for 7 consecutive days.

In [5]:
def consistentWorker(df, consecutive_days=7):
    workers = []

    for index, row in df.iterrows():
        if index > 0 and is_same_employee(row, df.iloc[index - 1]):
            days_diff = (row['Time'] - df.iloc[index - 1]['Time']).days
            if days_diff == 1:
                if len(workers) == 0 or workers[-1]['Name'] != row['Employee Name']:
                    # add worker to the list if shift diff is more than or equal to 1 day 
                    workers.append({'Name': row['Employee Name'],'Position':row['Position ID'], 'Days': 1})
                else:
                    workers[-1]['Days'] += 1
            elif days_diff > 1:
                # Reset consecutive days count if there is a gap
                workers[-1]['Days'] = 1
        else:
            workers.append({'Name': row['Employee Name'],'Position':row['Position ID'], 'Days': 1})
    # Filter workers who worked for the specified consecutive days
    result = [{worker['Name'],worker['Position']} for worker in workers if worker['Days'] >= consecutive_days]
    return result

# Example usage:
consecutive_workers = consistentWorker(df, consecutive_days=7)
for worker in consecutive_workers:
    print(worker)

{'WFS000462', 'XiWW, JAsMiW'}
